<a href="https://colab.research.google.com/github/ThunderBolt-07/The-Rescuer/blob/main/Accident_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import files
import cv2
import os
import numpy as np
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
processed_accidents_path='/content/drive/My Drive/accidents_gray'
processed_non_accidents_path='/content/drive/My Drive/non_accidents_gray'

In [ ]:
images_as_num=[]
labels=[]
for image in os.listdir(processed_accidents_path):
  image_gray=cv2.imread(os.path.join(processed_accidents_path,image),cv2.IMREAD_GRAYSCALE)

  images_as_num.append(cv2.GaussianBlur(image_gray, (5, 5), 0))
  labels.append(1)
images_as_num=images_as_num*2
labels=labels*2

In [ ]:
print(images_as_num)

[array([[228, 228, 229, ...,  53,  53,  53],
       [228, 228, 228, ...,  53,  53,  53],
       [227, 228, 228, ...,  52,  52,  52],
       ...,
       [ 51,  50,  49, ...,  39,  39,  39],
       [ 49,  48,  46, ...,  39,  39,  39],
       [ 48,  47,  45, ...,  39,  39,  39]], dtype=uint8), array([[115, 116, 117, ...,  26,  30,  31],
       [117, 118, 119, ...,  25,  29,  30],
       [120, 121, 122, ...,  23,  27,  27],
       ...,
       [ 13,  14,  16, ...,  16,  15,  14],
       [ 11,  12,  14, ...,  15,  13,  11],
       [ 11,  11,  13, ...,  14,  12,  10]], dtype=uint8), array([[210, 210, 210, ..., 224, 224, 224],
       [209, 209, 210, ..., 224, 224, 224],
       [203, 206, 209, ..., 224, 224, 224],
       ...,
       [ 30,  30,  30, ...,  41,  46,  49],
       [ 28,  28,  28, ...,  38,  38,  39],
       [ 28,  28,  28, ...,  38,  37,  36]], dtype=uint8), array([[166, 168, 171, ..., 174, 171, 169],
       [169, 171, 174, ..., 177, 173, 171],
       [173, 175, 178, ..., 181, 176, 

In [ ]:
for image in os.listdir(processed_non_accidents_path):
  image_gray=cv2.imread(os.path.join(processed_non_accidents_path,image),cv2.IMREAD_GRAYSCALE)
  images_as_num.append(cv2.GaussianBlur(image_gray, (5, 5), 0))
  labels.append(0)

In [ ]:
print(images_as_num)
print(len(labels))

356
356


In [ ]:
y=np.array(labels)
x=np.array(images_as_num)


In [ ]:
x.shape

(356, 224, 224)

In [ ]:
rgb_image_array = np.stack((x,) * 3, axis=-1)
rgb_image_array.shape


(356, 224, 224, 3)

In [ ]:
x=rgb_image_array


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
np.unique(y_train,return_counts=True)

(array([1]), array([284]))

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
model=tf.keras.applications.ResNet101(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None,
    classes=2,
    classifier_activation='sigmoid'
)
model.summary()

Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                      

In [ ]:
model.output

<KerasTensor: shape=(None, 7, 7, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [ ]:


base_model = tf.keras.applications.ResNet101(include_top=False, weights='imagenet', input_shape=(224, 224, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_4[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
cw = {0: 1.0, 1: 1.2}
model.fit(x_train,y_train,epochs=15,validation_data=(x_test,y_test),class_weight=cw)

Epoch 1/15
9/9 [==============================] - 17s 1s/step - loss: 0.0524 - accuracy: 0.9930 - val_loss: 1.1608e-23 - val_accuracy: 1.0000
Epoch 2/15
9/9 [==============================] - 8s 950ms/step - loss: 5.9507e-22 - accuracy: 1.0000 - val_loss: 3.5742e-30 - val_accuracy: 1.0000
Epoch 3/15
9/9 [==============================] - 9s 993ms/step - loss: 6.0657e-27 - accuracy: 1.0000 - val_loss: 5.5278e-33 - val_accuracy: 1.0000
Epoch 4/15
9/9 [==============================] - 8s 962ms/step - loss: 1.8083e-28 - accuracy: 1.0000 - val_loss: 3.2484e-34 - val_accuracy: 1.0000
Epoch 5/15
9/9 [==============================] - 9s 971ms/step - loss: 3.1335e-29 - accuracy: 1.0000 - val_loss: 9.5227e-35 - val_accuracy: 1.0000
Epoch 6/15
9/9 [==============================] - 8s 962ms/step - loss: 1.3229e-29 - accuracy: 1.0000 - val_loss: 5.6407e-35 - val_accuracy: 1.0000
Epoch 7/15
9/9 [==============================] - 8s 950ms/step - loss: 1.0100e-29 - accuracy: 1.0000 - val_loss: 4.52

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Predict on test data
# print(x_test)
y_pred = model.predict(x_test)

# Convert predictions to class labels
# For binary classification
y_pred_labels = (y_pred > 0.5).astype(int)

# For multi-class classification (use this line instead for multi-class)
# y_pred_labels = np.argmax(y_pred, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Compute and print classification report
report = classification_report(y_test, y_pred_labels)
print("Classification Report:")
print(report)


3/3 [==============================] - 4s 510ms/step
Confusion Matrix:
[[72]]
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        72

    accuracy                           1.00        72
   macro avg       1.00      1.00      1.00        72
weighted avg       1.00      1.00      1.00        72



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:
model.save(('/content/drive/My Drive/accident_detection_model.h5'))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install twilio
from twilio.rest import Client

# Twilio credentials
account_sid = 'your_account_sid'
auth_token = 'your_auth_token'
client = Client(account_sid, auth_token)

# Send an alert to authorities
message = client.messages.create(
    body="Accident detected! Please send help.",
    from_='+1234567890', # Your Twilio number
    to='+0987654321'     # Authority's phone number
)

print(message.sid)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 13.0 MB/s eta 0:00:00


TwilioRestException: HTTP 401 error: Unable to create record: Authentication Error - invalid username